In [12]:
import cv2
from deepface import DeepFace

# Load face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start capturing video
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the face ROI (Region of Interest) - use original BGR frame
        face_roi = frame[y:y + h, x:x + w]

        try:
            # Perform emotion analysis on the face ROI
            # Use skip detector since we already detected faces with OpenCV
            result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False, detector_backend='skip')

            # Determine the dominant emotion
            emotion = result[0]['dominant_emotion']

            # Draw rectangle around face and label with predicted emotion
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        except Exception as e:
            # If analysis fails, just draw the rectangle
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Real-time Emotion Detection', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [3]:
import cv2
from deepface import DeepFace

# 1. Load face cascade classifier (pre-trained model to find faces)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 2. Start capturing video from the webcam (Index 0)
cap = cv2.VideoCapture(0)

# --- FULL SCREEN SETUP START ---
window_name = 'Real-time Emotion Detection'
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
# --- FULL SCREEN SETUP END ---

while True:
    # 3. Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # 4. Convert frame to grayscale (Haar cascades need grayscale)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 5. Detect faces in the frame
    # scaleFactor=1.1: Image is reduced by 10% at each scale check (detects faces at different distances)
    # minNeighbors=5: Higher quality threshold (reduces false positives)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # 6. Extract the face Region of Interest (ROI)
        # We cut out just the face from the color frame to send to DeepFace
        face_roi = frame[y:y + h, x:x + w]

        try:
            # 7. Perform emotion analysis
            # detector_backend='skip': We skip DeepFace's built-in face detection because we already did it with OpenCV (faster)
            # enforce_detection=False: Prevents crash if DeepFace is unsure it's a face
            result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False, detector_backend='skip')

            # 8. Determine the dominant emotion
            emotion = result[0]['dominant_emotion']

            # 9. Draw rectangle and text (Red color)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        
        except Exception as e:
            # If DeepFace fails, we still draw a Green rectangle so you know a face was tracked
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 10. Display the resulting frame in the full-screen window
    cv2.imshow(window_name, frame)

    # 11. Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 12. Cleanup
cap.release()
cv2.destroyAllWindows()

In [2]:
import cv2
import tkinter as tk  # Used to get screen resolution
from deepface import DeepFace

# --- 1. DETECT SCREEN SIZE ---
# We create a hidden Tkinter window just to get the screen width/height
root = tk.Tk()
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
root.destroy()  # Close Tkinter immediately

# --- 2. SETUP CAMERA & MODEL ---
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

# Define the window name
window_name = 'Real-time Emotion Detection'

# Create a "Normal" window that can be resized
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

# FORCE Full Screen:
# Step A: Move window to top-left corner (0,0)
cv2.moveWindow(window_name, 0, 0)
# Step B: Resize window to exact screen dimensions
cv2.resizeWindow(window_name, screen_width, screen_height)
# Step C: Attempt to set the fullscreen property (removes borders/title bar)
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

print(f"Screen detected: {screen_width}x{screen_height}")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for faster face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_roi = frame[y:y + h, x:x + w]

        try:
            # Analyze emotion
            result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False, detector_backend='skip')
            emotion = result[0]['dominant_emotion']

            # Draw Red Box & Text
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        
        except Exception:
            # Draw Green Box if analysis fails
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show the frame
    cv2.imshow(window_name, frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Screen detected: 1536x864
